In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
#Read in genotype data

genotype_df = pd.read_csv("test_genotypes.csv")

In [3]:
#Take transpose and discard extraneous info

#X = genotype_df.T[4:]

#For test purposes use:
X = genotype_df

In [4]:
#DATA OWNER
#Set the parameters of the encryption context.

parms = EncryptionParameters(scheme_type.ckks)
poly_modulus_degree = 4096
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [35, 30, 35]))
#100-bit coeff modulus Q. 
#128 bit security for Q < 109 bits
scale = 2.0**30
context = SEALContext(parms)
#print_parameters(context)

#print(CoeffModulus.MaxBitCount(poly_modulus_degree))

ckks_encoder = CKKSEncoder(context)
slot_count = ckks_encoder.slot_count()
#print(f'Number of slots: {slot_count}')

keygen = KeyGenerator(context)
public_key = keygen.create_public_key()
secret_key = keygen.secret_key()
galois_keys = keygen.create_galois_keys()
relin_keys = keygen.create_relin_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)

In [5]:
#Prepare plaintexts from data.
def sample_to_pt(sample):
    
    pt_data = np.zeros((12,2048))
    for i in range(len(sample)):
        pt_data[i // 2048, i%2048] = sample[i]
    
    #Add extra feature, always equal to 1.0, to multiply by model intercepts
    pt_data[11,562] = 1.0
    
    return pt_data

In [6]:
#Batch and encrypt data

for m in range(len(X.index)):
    
    data_batch = sample_to_pt(np.concatenate((np.array(X.iloc[m]),np.array([1.0]))))
    #data_pt = []
    #data_ct = []

    for i in range(12):
        data_pt = ckks_encoder.encode(data_batch[i],scale)
        data_ct = encryptor.encrypt(data_pt)
        data_ct.save('data_ct_[%i,%i]'%(m,i))

In [7]:
#Save encryption context info


parms.save('encr_parms')
public_key.save('pubkey')
galois_keys.save('galkeys')
relin_keys.save('relinkeys')
pickle.dump(scale, open('scale','wb'))

secret_key.save('secretkey')